In [1]:
!pip install confluent-kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 71.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import json

csv_file = '/content/first_100_customers.csv'

df = pd.read_csv(csv_file)

df.head()




,customer_id,name,city,state,country,registration_date,is_active
0,0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False


In [4]:
json_records = df.to_dict(orient='records')

json_file = 'customers.json'

with open (json_file, 'w') as file:
  json.dump(json_records,file, indent=4)

print("File Converted to json")


File Converted to json


In [ ]:
# Required connection configs for Kafka producer, consumer, and admin
"bootstrap.servers"="pkc-921jm.us-east-2.aws.confluent.cloud:9092"
"security.protocol"="SASL_SSL"
"sasl.mechanisms"="PLAIN"
"sasl.username"="PNW22WFHBS7VCCQG"
"sasl.password"="cfltqjhbRMnHYAWjV96t6ZM5RZTaphdjFtK6eVtTBGgTVAa9Hs8j/PSDVBCTqP3w"
# Best practice for higher availability in librdkafka clients prior to 1.7
session.timeout.ms=45000

client.id=ccloud-python-client-420a5c94-4186-4fe4-92d6-31366f7ac8bf


In [5]:
from confluent_kafka import Producer
import json
import time

conf = {
    "bootstrap.servers":"pkc-921jm.us-east-2.aws.confluent.cloud:9092",
    "security.protocol":"SASL_SSL",
    "sasl.mechanisms":"PLAIN",
    "sasl.username":"PNW22WFHBS7VCCQG",
    "sasl.password":"cfltqjhbRMnHYAWjV96t6ZM5RZTaphdjFtK6eVtTBGgTVAa9Hs8j/PSDVBCTqP3w",
    "session.timeout.ms":45000,
    "client.id":"ccloud-python-client-420a5c94-4186-4fe4-92d6-31366f7ac8bf"
}

producer = Producer(conf)

In [6]:
topic = 'ecommerce'

with open('/content/customers.json', 'r') as file:
  customers_data = json.load(file)

  value = customers_data[0]
  key = value['customer_id']

  print(key,value)

0 {'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}


In [7]:
producer.produce(topic, key=key, value=value)

TypeError: a bytes-like object is required, not 'dict'

In [ ]:
type(str(value).encode("utf-8"))

In [10]:
producer.produce(topic, key = str(key).encode('utf-8'),value = str(value).encode('utf-8'))

In [12]:
def delivery_status(err,msg):
  if err:
    print(f'Message Delivery Failed: {err}')
  else:
    print(f'Message Delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}')


for record in customers_data:
  try:

    message_value = json.dumps(record)
    message_key = str(record['customer_id']).encode('utf-8')

    producer.produce (topic, key=message_key, value=message_value, callback = delivery_status)
    producer.poll(1)

  except Exception as e:
    print(f"Error Sending Messages: {e}")

producer.flush()

print("Messages sent to Kafka Succesfully")

Message Delivered to ecommerce [2] at offset 3
Message Delivered to ecommerce [2] at offset 4
Message Delivered to ecommerce [1] at offset 0
Message Delivered to ecommerce [1] at offset 1
Message Delivered to ecommerce [1] at offset 2
Message Delivered to ecommerce [1] at offset 3
Message Delivered to ecommerce [1] at offset 4
Message Delivered to ecommerce [0] at offset 1
Message Delivered to ecommerce [2] at offset 5
Message Delivered to ecommerce [0] at offset 2
Message Delivered to ecommerce [0] at offset 3
Message Delivered to ecommerce [0] at offset 4
Message Delivered to ecommerce [0] at offset 5
Message Delivered to ecommerce [2] at offset 6
Message Delivered to ecommerce [0] at offset 6
Message Delivered to ecommerce [1] at offset 5
Message Delivered to ecommerce [0] at offset 7
Message Delivered to ecommerce [2] at offset 7
Message Delivered to ecommerce [0] at offset 8
Message Delivered to ecommerce [1] at offset 6
Message Delivered to ecommerce [0] at offset 9
Message Deliv